In [2]:
import pandas as pd
import numpy as np

In [3]:
price_file = "../Price/FB.csv"
price = pd.read_csv(price_file, usecols=['Date', 'Adj Close'])

In [4]:
sec_file = "../SEC_8K/Data/8K data.h5"
sec = pd.read_hdf(sec_file)

In [5]:
twitter_file = "../Twitter/twitter.h5"
twitter = pd.read_hdf(twitter_file)

In [6]:
price = price.set_index("Date")
price.head()

,Adj Close
Date,
2012-05-18,38.230000
2012-05-21,34.029999
2012-05-22,31.000000
2012-05-23,32.000000
2012-05-24,33.029999


In [7]:
col = {}
for i in range(50):
    col[i] = "sec{}".format(i)

In [8]:
sec = pd.DataFrame(sec.values.tolist(), index=sec.index).rename(columns=col)
sec.head()

,sec0,sec1,sec2,sec3,sec4,sec5,sec6,sec7,sec8,sec9,...,sec40,sec41,sec42,sec43,sec44,sec45,sec46,sec47,sec48,sec49
2012-06-25,0.403492,-1.199431,0.132970,0.316648,0.481768,-0.637601,0.735545,0.100438,0.404877,-0.594432,...,-0.352549,0.947459,1.227568,0.193733,0.216422,0.761959,-0.039660,0.663948,0.280306,-0.958587
2012-07-26,0.317928,-0.670532,-0.414336,0.698862,0.525606,-0.713975,1.212491,-0.424508,0.420412,0.124043,...,-0.495426,0.870042,0.549239,0.554414,0.328707,0.566516,0.397477,0.560907,0.051546,-0.357353
2012-09-04,2.056731,-0.373491,1.769456,0.399185,1.954762,-2.903101,2.543088,0.454259,-1.388258,1.432291,...,-0.256293,-3.724535,0.688834,-0.186197,0.907559,-1.688571,-0.231034,1.781301,2.557888,2.704398
2012-10-15,2.022744,-2.139764,0.373905,0.351427,1.427031,-0.553096,1.560566,-0.128376,1.277749,0.429789,...,-1.616373,1.923072,1.246866,1.004811,-1.625501,1.093239,-0.253766,-0.261982,0.939612,0.685963
2012-10-23,0.883834,-0.712416,-0.221853,0.418001,0.406507,-0.800336,0.929981,-0.094012,0.710542,0.138464,...,-0.632370,0.976416,0.845908,0.888584,0.048708,0.588752,0.006392,0.608637,0.413090,-0.614922


In [9]:
twitter.head()

,sentiment
date,
2012-04-01,0.035836
2012-04-02,0.043281
2012-04-03,0.000000
2012-04-04,-0.004045
2012-04-05,0.085019


In [10]:
data = price.join(twitter).dropna().join(sec).fillna(0)

In [11]:
data['next_day_value'] = data['Adj Close'].shift(-1)

In [12]:
data = data.dropna()

In [13]:
data.head()

,Adj Close,sentiment,sec0,sec1,sec2,sec3,sec4,sec5,sec6,sec7,...,sec41,sec42,sec43,sec44,sec45,sec46,sec47,sec48,sec49,next_day_value
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-18,38.230000,-0.076487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.029999
2012-05-21,34.029999,0.043950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.000000
2012-05-22,31.000000,-0.182019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.000000
2012-05-23,32.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.029999
2012-05-24,33.029999,-0.126471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.910000


In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [15]:
train, test = train_test_split(data, test_size=0.3, shuffle=False)
train = train.copy()
test = test.copy()

In [16]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(train["Adj Close"].values.reshape(-1,1))
test["Adj Close"] = scaler.transform(test["Adj Close"].values.reshape(-1,1))
test["next_day_value"] = scaler.transform(test["next_day_value"].values.reshape(-1,1))
train["Adj Close"] = scaler.transform(train["Adj Close"].values.reshape(-1,1))
train["next_day_value"] = scaler.transform(train["next_day_value"].values.reshape(-1,1))

In [18]:
train

,Adj Close,sentiment,sec0,sec1,sec2,sec3,sec4,sec5,sec6,sec7,...,sec41,sec42,sec43,sec44,sec45,sec46,sec47,sec48,sec49,next_day_value
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-18,-0.638193,-0.076487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.712319
2012-05-21,-0.712319,0.043950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.765796
2012-05-22,-0.765796,-0.182019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.748147
2012-05-23,-0.748147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.729968
2012-05-24,-0.729968,-0.126471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.749735
2012-05-25,-0.749735,0.066473,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.803918
2012-05-29,-0.803918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.815390
2012-05-30,-0.815390,0.208429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.790505
2012-05-31,-0.790505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.823685


In [18]:
data.to_hdf("../agg_data.h5", key='df', mode='w')
train.to_hdf("../train_next_day.h5", key='df', mode='w')
test.to_hdf("../test_next_day.h5", key='df', mode='w')

In [19]:
from sklearn.externals import joblib
scaler_filename = "../scaler.save"
joblib.dump(scaler, scaler_filename) 

['../scaler.save']